# Q1

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import mode

def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

def cosine_similarity(point1, point2):
    dot_product = np.dot(point1, point2)
    magnitude_product = np.linalg.norm(point1) * np.linalg.norm(point2)
    return 1 - dot_product / magnitude_product if magnitude_product != 0 else 0

def jaccard_similarity(point1, point2):
    intersection = np.sum(np.minimum(point1, point2))
    union = np.sum(np.maximum(point1, point2))
    return 1 - intersection / union if union != 0 else 0

def make_cluster(data, centroids, distance_function=euclidean_distance):
    k = len(centroids)
    set_of_clusters = [[] for _ in range(k)]
    for d in data:
        min_dist = distance_function(centroids[0], d)
        assign = 0
        for i in range(1, k):
            curr_dist = distance_function(centroids[i], d)
            if curr_dist < min_dist:
                assign = i
                min_dist = curr_dist
        set_of_clusters[assign].append(d)
    return set_of_clusters

def compute_centroids(set_of_clusters):
    centroids = []
    for cluster in set_of_clusters:
        centroid = mean_distance(cluster)
        centroids.append(centroid)
    return centroids

def mean_distance(cluster):
    if not cluster:
        return None
    num_instances = len(cluster)
    num_attributes = len(cluster[0])
    means = [0] * num_attributes
    for instance in cluster:
        for i in range(num_attributes):
            means[i] += instance[i]
    for i in range(num_attributes):
        means[i] /= float(num_instances)
    return tuple(means)

def k_means(data, k, distance_function=euclidean_distance, max_iterations=100):
    iteration = 0
    # Step 1: Choosing k random centroids
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    set_of_clusters = [set() for _ in range(k)]
    #step 2: assigning points to centre and caluating new centroid
    while iteration < max_iterations:
        old_centroids = np.copy(centroids)
        set_of_clusters = make_cluster(data, centroids, distance_function)
        centroids = compute_centroids(set_of_clusters)
        #step 3: stop when centroids remain same
        if np.all(old_centroids == centroids):
            break
        iteration += 1
    return set_of_clusters, centroids

def calculate_sse(set_of_clusters, centroids, distance_function=euclidean_distance):
    sse = 0
    for i in range(len(set_of_clusters)):
        centroid = centroids[i]
        cluster = set_of_clusters[i]
        for point in cluster:
            sse += distance_function(point, centroid) ** 2
    return sse


# Load data and labels
data = pd.read_csv('kmeans_data/data.csv', header=None).values
labels = pd.read_csv('kmeans_data/label.csv', header=None).values.flatten()

# Number of unique labels/categories
categorical_values = len(np.unique(labels))

# Euclidean K-means
clustering_euclidean = k_means(data, k=categorical_values, distance_function=euclidean_distance)
clusters_euclidean, centroids_euclidean = clustering_euclidean
sse_euclidean = calculate_sse(clusters_euclidean, centroids_euclidean, distance_function=euclidean_distance)

print("SSE (Euclidean):", sse_euclidean)

# Cosine K-means
clustering_cosine = k_means(data, k=categorical_values, distance_function=cosine_similarity)
clusters_cosine, centroids_cosine = clustering_cosine
sse_cosine = calculate_sse(clusters_cosine, centroids_cosine, distance_function=cosine_similarity)

print("SSE (Cosine):", sse_cosine)

# Jaccard K-means
clustering_jaccard = k_means(data, k=categorical_values, distance_function=jaccard_similarity)
clusters_jaccard, centroids_jaccard = clustering_jaccard
sse_jaccard = calculate_sse(clusters_jaccard, centroids_jaccard, distance_function=jaccard_similarity)

print("SSE (Jaccard):", sse_jaccard)

SSE (Euclidean): 25656776184.834675
SSE (Cosine): 686.3912010012293
SSE (Jaccard): 3719.8085530237995


# Q2

In [42]:
def label_clusters(set_of_clusters, labels):
    cluster_labels = []
    assigned_labels = np.zeros(len(data)) 
    for cluster in set_of_clusters:
        cluster_array = np.array(cluster)
        cluster_indices = np.concatenate([np.where(np.all(data == np.array(list(cluster_set)), axis=1))[0] for cluster_set in cluster])
        # print(cluster_indices)
        if cluster_indices[0].size > 0:
            mode_label = mode(labels[cluster_indices])[0]
            cluster_labels.append(mode_label)
            assigned_labels[cluster_indices] = mode_label
        else:
            cluster_labels.append(None)
    # print(cluster_labels)
    return assigned_labels

def calculate_accuracy(predicted_labels, true_labels):
    correct_predictions = np.sum(predicted_labels == true_labels)
    total_points = len(true_labels)
    accuracy = correct_predictions / total_points
    return accuracy*100

# Applying the majority vote labels and calculating accuracy for each distance metric

# Euclidean K-means
euclidean_majority_vote = label_clusters(clusters_euclidean, labels)
accuracy_euclidean = calculate_accuracy(euclidean_majority_vote, labels)
print("Accuracy Euclidean: ", accuracy_euclidean, "%")

# Cosine K-means
cosine_majority_vote = label_clusters(clusters_cosine, labels)
accuracy_cosine = calculate_accuracy(cosine_majority_vote, labels)
print("Accuracy Cosine: ", accuracy_cosine, "%")

# Jaccard K-means
jaccard_majority_vote = label_clusters(clusters_jaccard, labels)
accuracy_jaccard = calculate_accuracy(jaccard_majority_vote, labels)
print("Accuracy Jaccard: ", accuracy_jaccard, "%")


Accuracy Euclidean:  54.83 %
Accuracy Cosine:  63.349999999999994 %
Accuracy Jaccard:  61.61 %


# Q3

In [45]:
import time

def k_means_with_stop(data, k, distance_function=euclidean_distance, max_iterations=100, stop_criteria=500):
    sse_values = []
    iteration = 0
    start_time = time.time()
    
    # Step 1: Choosing k random centroids
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    set_of_clusters = [set() for _ in range(k)]
    
    #step 2: assigning points to centre and caluating new centroid
    while iteration < max_iterations:
        old_centroids = np.copy(centroids)
        set_of_clusters = make_cluster(data, centroids, distance_function)
        centroids = compute_centroids(set_of_clusters)

        sse = calculate_sse(set_of_clusters, centroids, distance_function)
        sse_values.append(sse)
        
        #step 3: stop when centroids remain same OR when SSE value increases in next iteration OR when max iterations is reached (used in while loop)
        if (np.all(old_centroids == centroids) or (iteration > 2 and sse_values[-2] < sse_values[-1])):
            break
        iteration += 1
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return set_of_clusters, centroids, sse_values, elapsed_time

# Euclidean K-means
clustering_euclidean = k_means_with_stop(data, k=categorical_values, distance_function=euclidean_distance)
clusters_euclidean, centroids_euclidean, euclidean_sse_values, euclidean_time = clustering_euclidean
euclidean_iterations = len(euclidean_sse_values)

print("Number of iterations (Euclidean):", euclidean_iterations)
print("Time taken (Euclidean):", euclidean_time)

# Cosine K-means
clustering_cosine = k_means_with_stop(data, k=categorical_values, distance_function=cosine_similarity)
clusters_cosine, centroids_cosine, cosine_sse_values, cosine_time = clustering_cosine
cosine_iterations = len(cosine_sse_values)

print("Number of iterations (Cosine):", cosine_iterations)
print("Time taken (Cosine):", cosine_time)

# Jaccard K-means
clustering_jaccard = k_means_with_stop(data, k=categorical_values, distance_function=jaccard_similarity)
clusters_jaccard, centroids_jaccard, jaccard_sse_values, jaccard_time = clustering_jaccard
jaccard_iterations = len(jaccard_sse_values)

print("Number of iterations (Jaccard):", jaccard_iterations)
print("Time taken (Jaccard):", jaccard_time)


Number of iterations (Euclidean): 51
Time taken (Euclidean): 159.2000377178192
Number of iterations (Cosine): 26
Time taken (Cosine): 145.97199892997742
Number of iterations (Jaccard): 15
Time taken (Jaccard): 79.87520813941956


# Q4

In [47]:
#SEPARATE FUNCTIONS
import time

def k_means_with_stop_centroid(data, k, distance_function=euclidean_distance, max_iterations=100, stop_criteria=500):
    sse_values = []
    iteration = 0
    start_time = time.time()
    
    # Step 1: Choosing k random centroids
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    set_of_clusters = [set() for _ in range(k)]
    
    #step 2: assigning points to centre and caluating new centroid
    while iteration < max_iterations:
        old_centroids = np.copy(centroids)
        set_of_clusters = make_cluster(data, centroids, distance_function)
        centroids = compute_centroids(set_of_clusters)

        sse = calculate_sse(set_of_clusters, centroids, distance_function)
        sse_values.append(sse)
        
        #step 3: stop when centroids remain same
        if (np.all(old_centroids == centroids)):
            break
        iteration += 1
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return set_of_clusters, centroids, sse_values, elapsed_time

def k_means_with_stop_sse(data, k, distance_function=euclidean_distance, max_iterations=100, stop_criteria=500):
    sse_values = []
    iteration = 0
    start_time = time.time()
    
    # Step 1: Choosing k random centroids
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    set_of_clusters = [set() for _ in range(k)]
    
    #step 2: assigning points to centre and caluating new centroid
    while iteration < max_iterations:
        old_centroids = np.copy(centroids)
        set_of_clusters = make_cluster(data, centroids, distance_function)
        centroids = compute_centroids(set_of_clusters)

        sse = calculate_sse(set_of_clusters, centroids, distance_function)
        
        #step 3: stop when SSE value increases in next iteration
        if (iteration > 1 and sse_values[-1] < sse):
            break
            
        sse_values.append(sse)
        iteration += 1
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return set_of_clusters, centroids, sse_values, elapsed_time
    
def k_means_with_stop_iteration(data, k, distance_function=euclidean_distance, max_iterations=100, stop_criteria=500):
    sse_values = []
    iteration = 0
    start_time = time.time()
    
    # Step 1: Choosing k random centroids
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    set_of_clusters = [set() for _ in range(k)]
    
    #step 2: assigning points to centre and caluating new centroid
    while iteration < max_iterations:
        old_centroids = np.copy(centroids)
        set_of_clusters = make_cluster(data, centroids, distance_function)
        centroids = compute_centroids(set_of_clusters)

        sse = calculate_sse(set_of_clusters, centroids, distance_function)
        sse_values.append(sse)
        
        #stop when max iterations is reached (used in while loop)
        iteration += 1
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return set_of_clusters, centroids, sse_values, elapsed_time

In [48]:
#STOP - No chnage in centroid

# Euclidean K-means
clustering_euclidean = k_means_with_stop_centroid(data, k=categorical_values, distance_function=euclidean_distance)
clusters_euclidean, centroids_euclidean, euclidean_sse_values, euclidean_time = clustering_euclidean
euclidean_iterations = len(euclidean_sse_values)

print("SSE (Euclidean):", euclidean_sse_values[euclidean_iterations-1])
print("Number of iterations (Euclidean):", euclidean_iterations)
print("Time taken (Euclidean):", euclidean_time)

# Cosine K-means
clustering_cosine = k_means_with_stop_centroid(data, k=categorical_values, distance_function=cosine_similarity)
clusters_cosine, centroids_cosine, cosine_sse_values, cosine_time = clustering_cosine
cosine_iterations = len(cosine_sse_values)

print("SSE (Cosine):", cosine_sse_values[cosine_iterations-1])
print("Number of iterations (Cosine):", cosine_iterations)
print("Time taken (Cosine):", cosine_time)

# Jaccard K-means
clustering_jaccard = k_means_with_stop_centroid(data, k=categorical_values, distance_function=jaccard_similarity)
clusters_jaccard, centroids_jaccard, jaccard_sse_values, jaccard_time = clustering_jaccard
jaccard_iterations = len(jaccard_sse_values)

print("SSE (Jaccard):", jaccard_sse_values[jaccard_iterations-1])
print("Number of iterations (Jaccard):", jaccard_iterations)
print("Time taken (Jaccard):", jaccard_time)

SSE (Euclidean): 25407000548.31759
Number of iterations (Euclidean): 67
Time taken (Euclidean): 207.71611905097961
SSE (Cosine): 682.7780497511749
Number of iterations (Cosine): 100
Time taken (Cosine): 586.1677749156952
SSE (Jaccard): 3719.2888934910698
Number of iterations (Jaccard): 58
Time taken (Jaccard): 328.01897382736206


In [49]:
#STOP - SSE increase

# Euclidean K-means
clustering_euclidean = k_means_with_stop_sse(data, k=categorical_values, distance_function=euclidean_distance)
clusters_euclidean, centroids_euclidean, euclidean_sse_values, euclidean_time = clustering_euclidean
euclidean_iterations = len(euclidean_sse_values)

print("SSE (Euclidean):", euclidean_sse_values[euclidean_iterations-1])
print("Number of iterations (Euclidean):", euclidean_iterations)
print("Time taken (Euclidean):", euclidean_time)

# Cosine K-means
clustering_cosine = k_means_with_stop_sse(data, k=categorical_values, distance_function=cosine_similarity)
clusters_cosine, centroids_cosine, cosine_sse_values, cosine_time = clustering_cosine
cosine_iterations = len(cosine_sse_values)

print("SSE (Cosine):", cosine_sse_values[cosine_iterations-1])
print("Number of iterations (Cosine):", cosine_iterations)
print("Time taken (Cosine):", cosine_time)

# Jaccard K-means
clustering_jaccard = k_means_with_stop_sse(data, k=categorical_values, distance_function=jaccard_similarity)
clusters_jaccard, centroids_jaccard, jaccard_sse_values, jaccard_time = clustering_jaccard
jaccard_iterations = len(jaccard_sse_values)

print("SSE (Jaccard):", jaccard_sse_values[jaccard_iterations-1])
print("Number of iterations (Jaccard):", jaccard_iterations)
print("Time taken (Jaccard):", jaccard_time)

SSE (Euclidean): 25491396453.425354
Number of iterations (Euclidean): 100
Time taken (Euclidean): 337.459431886673
SSE (Cosine): 692.4336831031989
Number of iterations (Cosine): 28
Time taken (Cosine): 175.65974116325378
SSE (Jaccard): 3661.15868094507
Number of iterations (Jaccard): 21
Time taken (Jaccard): 133.02547788619995


In [50]:
#STOP - Max iterations

# Euclidean K-means
clustering_euclidean = k_means_with_stop_iteration(data, k=categorical_values, distance_function=euclidean_distance)
clusters_euclidean, centroids_euclidean, euclidean_sse_values, euclidean_time = clustering_euclidean
euclidean_iterations = len(euclidean_sse_values)

print("SSE (Euclidean):", euclidean_sse_values[euclidean_iterations-1])
print("Number of iterations (Euclidean):", euclidean_iterations)
print("Time taken (Euclidean):", euclidean_time)

# Cosine K-means
clustering_cosine = k_means_with_stop_iteration(data, k=categorical_values, distance_function=cosine_similarity)
clusters_cosine, centroids_cosine, cosine_sse_values, cosine_time = clustering_cosine
cosine_iterations = len(cosine_sse_values)

print("SSE (Cosine):", cosine_sse_values[cosine_iterations-1])
print("Number of iterations (Cosine):", cosine_iterations)
print("Time taken (Cosine):", cosine_time)

# Jaccard K-means
clustering_jaccard = k_means_with_stop_iteration(data, k=categorical_values, distance_function=jaccard_similarity)
clusters_jaccard, centroids_jaccard, jaccard_sse_values, jaccard_time = clustering_jaccard
jaccard_iterations = len(jaccard_sse_values)

print("SSE (Jaccard):", jaccard_sse_values[jaccard_iterations-1])
print("Number of iterations (Jaccard):", jaccard_iterations)
print("Time taken (Jaccard):", jaccard_time)

SSE (Euclidean): 25538760757.619717
Number of iterations (Euclidean): 100
Time taken (Euclidean): 355.1627562046051
SSE (Cosine): 684.3100239771878
Number of iterations (Cosine): 100
Time taken (Cosine): 632.6668038368225
SSE (Jaccard): 3661.514522484269
Number of iterations (Jaccard): 100
Time taken (Jaccard): 620.5958359241486
